# Введение в обработку естественного языка
# Урок 13. Модель BERT и GPT

# <font color='red'>Практическое задание</font>

### Домашнее задание к уроку 13

__Задание__

1. Взять датасет<br>
https://huggingface.co/datasets/merionum/ru_paraphraser<br>
решить задачу парафраза

2. (дополнительно необязательная задача)на выбор взять<br>
https://huggingface.co/datasets/sberquad<br>
https://huggingface.co/datasets/blinoff/medical_qa_ru_data<br>
натренировать любую модель для вопросно ответной системы<br>
как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

Выполнил ___Соковнин ИЛ___

# Загрузка данных

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 35.8 MB/s 
     |████████████████████████████████| 1.2 MB 59.4 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 596 kB 68.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 18.3 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 1.1 MB 63.0 MB/s 
     |████████████████████████████████| 140 kB 73.0 MB/s 
     |████████████████████████████████| 127 kB 71.0 MB/s 
     |████████████████████████████████| 144 kB 71.7 MB/s 
     |████████████████████████████████| 271 kB 71.0 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Using custom data configuration merionum--ru_paraphraser-4bf06233b6bfe0ef


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4bf06233b6bfe0ef/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [5]:
corpus['train']

Dataset({
    features: ['id', 'paraphrases'],
    num_rows: 1725393
})

In [6]:
corpus['train'][0:3]

{'id': ['0', '1', '2'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем'],
  ['В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото)',
   'Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства',
   'В Индии женщины встали живой стеной длиной 620 км'],
  ['Томас Бах: "Олимпийский комитет России понес достаточное наказание"',
   'Президент МОК Бах: Россия уже достаточно наказана за допинг',
   'Глава МОК заявил, что ОКР понес наказание за манипуляции с пробами на ОИ в Сочи']]}

In [7]:
corpus['train']['paraphrases'][0]

['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
 'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']

In [8]:
corpus['train']['paraphrases'][0][0]

'Птичий праздник: Что такое "куриное рождество" и кто его отмечает'

In [9]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ea9d9a76fa9ddf4e68e7902d65b994cfc0712d888e89df216b9c5cbcdc80c348
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Используем предобученную модель: cointegrated/rut5-base-paraphraser

https://huggingface.co/cointegrated/rut5-base-paraphraser

In [10]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda();
model.eval();

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(paraphrase('Каждый охотник желает знать, где сидит фазан.'))
# Все охотники хотят знать где фазан сидит.

Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

Все охотники хотят знать где фазан сидит.


# Используем предобученную модель: cointegrated/rubert-base-cased-dp-paraphrase-detection

https://huggingface.co/cointegrated/rubert-base-cased-dp-paraphrase-detection

In [11]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
        
    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"
        
    return [text1, text2, p[0], p[1], paraphrase]


print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
# [0.7056226 0.2943774]
print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))
# [0.16524374 0.8347562 ]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

['Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода', 0.7056231, 0.29437694, 'non']
['Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась', 0.1652439, 0.83475614, 'paraphrase']


In [12]:
def compare(text1, text2):
    """  """

    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    
    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    paraphrase = "non"

    if p[1] > 0.5:
        paraphrase = "paraphrase"
    
    # row = {'текст1': text1, 'текст2': text2, 'вероятность non-paraphrase': p[0], 'вероятность paraphrase': p[1], 'paraphrase': paraphrase}
    # return row
    return [text1, text2, p[0], p[1], paraphrase]

In [13]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.width', 500)


In [15]:
result = []

result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода')) 
result.append(compare('Цены на нефть восстанавливаются', 'Парламент Словакии поблагодарил народы'))
result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице прекрасная погода')) 
n=5
for i in range(n):
  text1 = corpus['train']['paraphrases'][i][0]
  text2 =corpus['train']['paraphrases'][i][1]
  result.append(compare(text1, text2)) 



col_names =  ['текст1', 'текст2', 'вероятность non-paraphrase', 'вероятность paraphrase', 'paraphrase']
df = pd.DataFrame(result, columns=col_names)
df.head(10)

,текст1,текст2,вероятность non-paraphrase,вероятность paraphrase,paraphrase
0,Сегодня на улице хорошая погода,Сегодня на улице отвратительная погода,0.705623,0.294377,non
1,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы,0.944955,0.055045,non
2,Сегодня на улице хорошая погода,Сегодня на улице прекрасная погода,0.035242,0.964758,paraphrase
3,"Птичий праздник: Что такое ""куриное рождество"" и кто его отмечает",Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем,0.157879,0.842121,paraphrase
4,В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото),"Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства",0.018209,0.981791,paraphrase
5,"Томас Бах: ""Олимпийский комитет России понес достаточное наказание""",Президент МОК Бах: Россия уже достаточно наказана за допинг,0.028516,0.971484,paraphrase
6,Россия не представила WADA данные по пробам в срок и может быть наказана,WADA: Россия не предоставила нам доступ к допинг-пробам,0.058580,0.941420,paraphrase
7,Антидопинговое агентство США призвало ВАДА отстранить российских спортсменов от соревнований,Комиссия спортсменов WADA потребовала отстранить РУСАДА,0.428032,0.571968,paraphrase
